## 0. Libarary 불러오기 및 경로설정

In [1]:
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

In [2]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'

## 1. Model 정의

In [3]:
class MyModel(nn.Module):
    def __init__(self, num_classes: int = 1000):
        super(MyModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(64, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

## 2. Test Dataset 정의

In [4]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

## 3. Inference

In [8]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
#model = MyModel(num_classes=18).to(device)

#model.eval()

model_path = '/opt/ml/code/'
models = ["0831Fold_1_swin_large_patch4_window7_224_batch32_lr0.0001_89.71.pt",
"0831Fold_2_swin_large_patch4_window7_224_batch32_lr0.0001_93.46.pt",
"0831Fold_3_swin_large_patch4_window7_224_batch32_lr0.0001_90.14.pt",
"0831Fold_4_swin_large_patch4_window7_224_batch32_lr0.0001_85.46.pt",
"0831Fold_5_swin_large_patch4_window7_224_batch32_lr0.0001_86.13.pt"]
models = [torch.load(os.path.join(model_path, model)).eval() for model in models]

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.

# 모델 하나
if len(models) == 1:
    model = models[0]
    all_predictions = []
    for images in tqdm(loader):
        with torch.no_grad():
            images = images.to(device)
            pred = model(images)
            pred = pred.argmax(dim=-1)
            all_predictions.extend(pred.cpu().numpy())
    submission['ans'] = all_predictions
else:

    all_predictions = []


    for images in tqdm(loader):
        with torch.no_grad():
            images = images.to(device)
            npred = torch.zeros(images.size(0), 18)
            for model in models:
                outputs = model(images)

                npred = torch.add(npred,outputs.cpu())

            npred = npred / len(models)

            pred = npred.argmax(dim=-1)

            all_predictions.extend(pred.cpu().numpy())

    submission['ans'] = all_predictions
    

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 31.75 GiB total capacity; 1.81 GiB already allocated; 7.50 MiB free; 1.94 GiB reserved in total by PyTorch)